This notebook runs Select@10 and Match@10. 

Both of these involve measuring similarity between row pairs between two dataframes. This similarity can be measure in a supervised manner wherein, e.g. a logistic model re-weights distances according to learned importances, or they may be measured in an unsupervised manner, e.g. by an off-the-shelf distance metric.

Match performs the additional step of picking node pairs such that these similarities are globally optimized

In [1]:
from datastack.ml import bestmatch
reload(bestmatch)
from datastack.ml import bestmatch as bm
from datastack.dbs.rdb import RosettaDBMongo
import model_utils as mod


/Users/cmaher/miniconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
SELECT_N = 10

In [3]:
db = RosettaDBMongo(host="rosetta.hli.io")
db.initialize(namespace="hg38_noEBV")

In [4]:
col_key = u'.*2dface.v1.visit1.NNOF128.v2'
df = db.query(db.find_keys(col_key, regex=True) + ['ds.index.client_subject_id', 'ds.index.sample_key', 'ds.index.ProjectID'], with_nans=False)
print df.shape

(997, 5)


In [5]:
df.head()
#print "{}".format(df['dynamic.FACE_P.2dface.v1.visit1.NNOF128.v2'])

,ds.index.ProjectID,ds.index.client_subject_id,ds.index.sample_key,dynamic.FACE_P.2dface.v1.visit1.NNOF128.v2,dynamic.FACE.2dface.v1.visit1.NNOF128.v2
564d2159c37a455a0ca36187,FACE,15-0051,77403_187522789,"[0.037772425712073295, 0.011718937192791101, 0...","[-0.03861616179347, 0.036061447113752, -0.0234..."
564d2159c37a455a0ca36016,FACE,15-0107,77788_187522765,"[0.03844781293134461, 0.017844150095531974, 0....","[0.030196761712432, 0.083207756280899, -0.0036..."
564d2159c37a455a0ca36056,FACE,15-0179,81825_187522726,"[0.034422556122900613, 0.015137199318548143, 0...","[0.139982894063, 0.1503836363554, 0.0131916161..."
564d2159c37a455a0ca35de4,FACE,15-1072,89030_176530919,"[0.019610622707623135, 0.024768844623360708, -...","[0.084498710930347, 0.01313541457057, -0.03572..."
564d2159c37a455a0ca360c9,FACE,15-0156,78206_187522257,"[-0.0066136576455728645, -0.008273379388993838...","[0.10006500780582, 0.076437957584858, 0.069531..."


In [6]:
packed_df = mod.PackedFrame(df)
df_unpacked = packed_df.unpack()

dynamic.FACE_P.2dface.v1.visit1.NNOF128.v2
dynamic.FACE.2dface.v1.visit1.NNOF128.v2
Found 2 packed columns


In [12]:
pred_cols = [xx for xx in df_unpacked.columns if "FACE_P" in xx]
obs_cols = [xx.replace("FACE_P", "FACE") for xx in pred_cols]

In [14]:
# You can specify a number of models here
model = bm.WeightedNearestNeighbor()

# DistSelect accepts all metrics recognized by scipy.spatial.distance
# http://docs.scipy.org/doc/scipy/reference/spatial.distance.html
# You can also pass in a user-defined distance function
# model = bm.DistSelect(metric='cosine')

In [15]:
eval_sel = bm.EvalSelect(df_unpacked, model,
                         to_cols=obs_cols,
                         from_cols=pred_cols,
                         n_lineups=100,
                         select_n=SELECT_N
                        )

Done fetching lineups


In [17]:
eval_sel.fit()
eval_sel.predict()

(828, 128) (828, 128) (100000, 128)


match            0.422
select           0.383
select_unique    0.395
match_query      0.420
select_query     0.420
dtype: float64